In [ ]:
import pandas as pd
import anndata
import scanpy as sc
import seaborn as sb
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
#import ace_tools as tools

In [ ]:
#pip install ace_tools

In [ ]:
#print(dir(ace_tools))

## Testing Scanpy library

In [ ]:
#df = pd.read_csv('batch_corrected_expression_test.csv', index_col=0)
df = pd.read_csv('batch_corrected_expression.csv', index_col=0)
df

In [ ]:
df = df.drop(columns=['sym', 'ensembl_peptide_id'])
df

In [ ]:
# Ensure 'PRODUCT-TG' is a separate column, it is assumed to be in the last column
df = df.T
target_variable = df['PRODUCT-TG']  # Assuming PRODUCT-TG is the last column
target_variable.index.name = 'sample_id'
expression_data = df.drop(columns=['PRODUCT-TG'])  # Drop PRODUCT-TG column from expression data

In [ ]:
expression_data

In [ ]:
target_variable

In [ ]:
# Step 1: Load the MANIFEST.txt file into a DataFrame
manifest_df = pd.read_csv('MANIFEST.txt', sep='\t')  # Adjust the file path if needed
# Step 2: Assuming the MANIFEST file has columns 'Sample' and 'description', we set 'Sample' as the index
manifest_df.set_index('Sample', inplace=True)

manifest_df.head()

In [ ]:
target_variable_df = pd.Series(target_variable, name='PRODUCT-TG')
target_variable_df = target_variable_df.to_frame()
target_variable_df.index.name = 'sample_id'
#df_abc.rename(columns={'PRODUCT-TG':'value'}, inplace=True)
target_variable_df.head()

In [ ]:
# Step 3: Merge with the RNA-seq DataFrame
# Assuming 'df' is the DataFrame containing your RNA-seq data with sample IDs as the index
target_variable_obs = target_variable_df.join(manifest_df[['Experiment','description']], how='left')
target_variable_obs['platform'] = target_variable_obs['description'].str.extract(r'(HiSeq|NovaSeq|NextSeq)')
# Step 4: Display the resulting DataFrame
print(target_variable_obs.head())

In [ ]:
#target_variable.rename(columns={0:'sample', 1:'value'}, inplace=True)
#df = df.rename(columns={'target_variable': 'PRODUCT_TG'})
#target_variable.head()

In [ ]:
target_variable.index

In [ ]:
# Create AnnData object: Use the expression matrix and add the target variable as metadata
adata = sc.AnnData(X=expression_data)
#adata.obs['PRODUCT-TG'] = target_variable
adata.obs = target_variable_obs
print(adata.X)
print(adata.obs['PRODUCT-TG'])

In [ ]:
adata.obs

In [ ]:
# Annotate the data sets
print(adata.obs['Experiment'].value_counts())
print('')
print(adata.obs['platform'].value_counts())
print('')

In [ ]:
adata.shape

In [ ]:
adata.var

In [ ]:
adata.var_names

In [ ]:
# Quantity of genes expression by samples
adata.obs['n_genes'] = (adata.X > 0).sum(1)
adata.obs

In [ ]:
#Plot with the genes quantity
p3 = sb.histplot(adata.obs['n_genes'], kde=False, bins=20)
plt.show()

In [ ]:
#pip install --quiet scvi-tools[tutorials]
#!pip install scvi

In [ ]:
#import scvi

# Load the dataset without setting up for scvi-tools
#adata = scvi.data.spleen_lymph_cite_seq(run_setup_anndata=False)

# Display the first 5 rows and columns of adata.X
#print(adata.X[:5, :5])  # First 5 cells and first 5 genes


In [ ]:
product_tg_expression = adata.obs['PRODUCT-TG'].values.flatten()

In [ ]:
# Compute correlation for each gene with 'PRODUCT-TG'
correlation_values = []
p_values = []

for gene in adata.var_names:
    gene_expression = adata[:, gene].X.flatten()
    
    # Check if the gene expression is constant across samples
    if np.all(gene_expression == gene_expression[0]):  # All values are the same
        correlation_values.append(0)  # Assign 0 for later filtering
        p_values.append(0)
    else:
        corr, p_val = stats.spearmanr(gene_expression, product_tg_expression)
        correlation_values.append(corr)
        p_values.append(p_val)

In [ ]:
# Store results in AnnData
adata.var['Spearman_Correlation'] = correlation_values
adata.var['P_Value'] = p_values
adata.var

### Ranking genes

In [ ]:
# Define a small constant to avoid division by zero
epsilon = 1e-10  # Prevents division by zero

# Compute ranking score (higher absolute correlation & lower p-value)
adata.var['Rank_Score'] = adata.var['Spearman_Correlation'].abs() / (adata.var['P_Value'] + epsilon)

# Sort by rank score
ranked_genes = adata.var.sort_values(by='Rank_Score', ascending=False)


In [ ]:
ranked_genes

In [ ]:
# Select top genes for visualization
top_genes = ranked_genes.head(5).index.tolist()

# Violin plot for top correlated genes
sc.pl.violin(adata, keys=top_genes, jitter=0.4, multi_panel=True)

In [ ]:
product_tg_expression

In [ ]:
adata.obs.index

In [ ]:
# Reset index to move 'sample_id' from index to a regular column
adata.obs = adata.obs.reset_index()

In [ ]:
# Now that 'sample_id' is a regular column, use it in groupby
sc.pl.heatmap(adata, var_names=top_genes, groupby='sample_id', cmap='viridis', standard_scale='var', figsize=(12, 10), show_gene_labels=True)

In [ ]:
sc.pl.heatmap(
    adata,
    var_names=top_genes,  # List of top genes
    groupby='sample_id',  # Group by sample ID
    cmap='viridis',  # Better contrast colormap
    standard_scale='var',  # Normalize expression levels
    swap_axes=True,  # Flip x and y axes for better label readability
    figsize=(12, 10),  # Increase figure size
    #dendrogram=True,  # Add hierarchical clustering for better grouping
    show_gene_labels=True  # Ensure gene labels are visible
)


### Correlations

In [ ]:
# Compute correlation of each gene with the target variable 'PRODUCT-TG'
correlation_scores = expression_data.corrwith(target_variable)
print(correlation_scores)

In [ ]:
# Rank genes by their correlation with the target
ranked_genes = correlation_scores.sort_values(ascending=False)

# Show the top 10 genes most correlated with PRODUCT-TG
print(ranked_genes.head(10))

In [ ]:
import matplotlib.pyplot as plt

# Plot the top X0 most correlated genes
ranked_genes.head(50).plot(kind='bar', figsize=(10, 6))
plt.title("Top 50 Genes Correlated with PRODUCT-TG")
plt.ylabel("Correlation")
plt.xlabel("Gene")
plt.show()


## Regression análisis

In [ ]:
from sklearn.linear_model import LinearRegression

# Prepare the data for regression (ensure you don't use PRODUCT-TG in X)
X = expression_data  # Expression data for genes
y = target_variable  # The continuous target variable PRODUCT-TG

In [ ]:
# Create and fit a linear regression model
model = LinearRegression()
model.fit(X, y)

In [ ]:
# Get the coefficients for each gene (ranking by impact on the target)
gene_coefficients = pd.Series(model.coef_, index=X.columns)

In [ ]:
gene_coefficients

In [ ]:
# Rank genes by the absolute value of the coefficients
ranked_genes_regression = gene_coefficients.abs().sort_values(ascending=False)

# Show the top 10 genes that have the greatest impact on PRODUCT-TG
print(ranked_genes_regression.head(10))

In [ ]:
# Plot the ranking of the top X0 genes based on their impact (absolute coefficient value)
plt.figure(figsize=(10, 6))
ranked_genes_regression.head(50).plot(kind='bar', color='skyblue')
plt.title('Top 50 Genes Ranked by Impact on PRODUCT-TG (Regression Coefficients)')
plt.xlabel('Gene')
plt.ylabel('Absolute Coefficient Value')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# pip install pingouin

## Visualizing the Relationship Between Genes and PRODUCT-TG

In [ ]:
import matplotlib.pyplot as plt

# Plot the relationship between a gene's expression and PRODUCT-TG
gene_of_interest = 'ENSCGRT00001000822'  # Replace with actual gene name
plt.scatter(expression_data[gene_of_interest], target_variable)
plt.xlabel(f'{gene_of_interest} Expression')
plt.ylabel('PRODUCT-TG')
plt.title(f'Scatter plot of {gene_of_interest} vs PRODUCT-TG')
plt.show()


In [ ]:
# Show the top 10 genes that have the greatest impact on PRODUCT-TG
print(ranked_genes_regression.head(10))

In [ ]:
# Step 4: Perform Gene Set Enrichment Analysis (GSEA)
# Select the top-ranked genes (let's take the top 100)
top_genes = ranked_genes_regression.index[:100]